## Librerías

In [23]:
%%capture
%run "../../recurrentes.ipynb"

In [24]:
%%capture
%run "../../funciones.ipynb"

In [25]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [26]:
pd.set_option('display.max_rows', 1)

In [27]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l

data = pd.read_parquet(dataset_clase_ternaria_l)



## Reformateo y limpieza

In [28]:
data['tmobile_app'] = pd.to_numeric(data['tmobile_app'], errors='coerce')  # Convierte no numéricos a NaN
data['tmobile_app'] = data['tmobile_app'].astype('bool')  # Llena los NaN y convierte a booleano

data['cmobile_app_trx'] = pd.to_numeric(data['cmobile_app_trx'], errors='coerce')  # Convierte no numéricos a NaN
data['cmobile_app_trx'] = data['cmobile_app_trx'].astype('bool')  # Llena los NaN y convierte a booleano

In [29]:
data = data.drop(columns=['cprestamos_personales','mprestamos_personales'])


In [30]:
data = data.drop(columns=['internet',
 'mcomisiones_mantenimiento',
 'Master_Fvencimiento',
 'mrentabilidad_annual',
 'mcomisiones_otras',
 'mcomisiones',
 'mtarjeta_visa_consumo',
 'mtransferencias_recibidas'])

## Valores null por variable

In [31]:
# 3- parametros para calcular ganancia del modelo
ganancia_acierto = 273000
costo_estimulo = 7000
data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

# 4 - binarización de la clase ternaria
data['clase_binaria'] = np.where(data['clase_ternaria']=='BAJA+2', 1, 0)

# 5 Especificar mes de train y test
df_train = data[data['foto_mes']<=202106]
df_test = data[data['foto_mes']==202108]
clase_peso = data['clase_peso']
X_train = data.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =data['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']
w_train = data.loc[X_train.index, 'clase_peso']


# Ingresar mes train y mes score
train_data = data[data['foto_mes'] <= 202107]
score_data = data[data['foto_mes'] == 202109]

train_null_percentage = train_data.isnull().mean() * 100
score_null_percentage = score_data.isnull().mean() * 100

comparison_df = pd.DataFrame({'Train Null Percentage': train_null_percentage, 'Score Null Percentage': score_null_percentage})
comparison_df['diff'] = (comparison_df['Score Null Percentage'] - comparison_df['Train Null Percentage']).abs()

comparison_df_sorted = comparison_df.sort_values('diff', ascending=False)
pd.set_option('display.max_rows', None)
comparison_df_sorted



,Train Null Percentage,Score Null Percentage,diff
clase_ternaria,0.000000,100.000000,100.000000
Master_mconsumototal,60.134993,56.999952,3.135042
Master_mconsumospesos,60.134993,56.999952,3.135042
Master_cconsumos,60.134993,56.999952,3.135042
Master_mpagosdolares,60.134993,56.999952,3.135042
Master_mpagospesos,60.134993,56.999952,3.135042
Master_madelantodolares,60.134993,56.999952,3.135042
Master_madelantopesos,60.134993,56.999952,3.135042
Master_mconsumosdolares,60.134993,56.999952,3.135042
Master_cadelantosefectivo,60.134993,56.999952,3.135042


## Análisis drifting con PSI

In [32]:
%%capture
psi_results = []
for mes in train_data['foto_mes'].unique():
  for column in train_data.columns:
    if column not in ['foto_mes', 'clase_ternaria']:
      train_variable = train_data[column]
      score_variable = score_data[column]
      psi_value = psi(train_variable, score_variable)
      mes = mes 
      psi_results.append({'feature': column, 'psi': psi_value, 'mes':mes})

psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)


In [33]:
lista_pci = psi_df[psi_df['psi'] >= 0.2]['feature'].to_list()

In [34]:
lista_pci = ['Visa_mlimitecompra',
 'mcajeros_propios_descuentos',
 'Master_fultimo_cierre',
 'Visa_Finiciomora',
 'mprestamos_prendarios',
 'Visa_madelantopesos',
 'Visa_fultimo_cierre',
 'mpayroll2',
 'Master_Finiciomora',
 'mpayroll',
 #deaca para arriba infinito 
 'internet',
 'mcomisiones_mantenimiento',
 'Master_Fvencimiento',
 'mrentabilidad_annual',
 'mcomisiones_otras',
 'mcomisiones',
 'mtarjeta_visa_consumo',
 'mtransferencias_recibidas']

In [35]:
f = psi_df['psi'] > 0.2
lista_variables_con_drift = psi_df.loc[f, 'feature'].to_list()

In [36]:
lista_variables_con_drift

['mcajeros_propios_descuentos',
 'mcajeros_propios_descuentos',
 'mprestamos_prendarios',
 'mprestamos_prendarios',
 'mpayroll',
 'mpayroll2',
 'Visa_fultimo_cierre',
 'Visa_madelantopesos',
 'mcajeros_propios_descuentos',
 'Visa_mlimitecompra',
 'mcajeros_propios_descuentos',
 'Visa_Finiciomora',
 'mpayroll2',
 'mprestamos_prendarios',
 'Master_Finiciomora',
 'mpayroll',
 'mpayroll2',
 'mpayroll2',
 'Master_fultimo_cierre',
 'mpayroll',
 'Visa_fultimo_cierre',
 'mcajeros_propios_descuentos',
 'mpayroll',
 'mpayroll2',
 'Visa_mlimitecompra',
 'Visa_madelantopesos',
 'Visa_Finiciomora',
 'Master_Finiciomora',
 'mprestamos_prendarios',
 'Visa_mlimitecompra',
 'mcajeros_propios_descuentos',
 'mpayroll2',
 'mpayroll',
 'Visa_madelantopesos',
 'Visa_mlimitecompra',
 'Visa_fultimo_cierre',
 'mprestamos_prendarios',
 'Visa_fultimo_cierre',
 'mpayroll',
 'mpayroll2',
 'mpayroll',
 'mcajeros_propios_descuentos',
 'mpayroll2',
 'mpayroll',
 'mprestamos_prendarios',
 'mcajeros_propios_descuentos'

In [37]:
variable_name = 'Master_Finiciomora'
expected = train_data[variable_name]
actual = score_data[variable_name]

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=10, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

print(f'Cortes en {variable_name}: {breakpoints}')
expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

print(f'Frecuencia Esperada: {expected_counts}')
print(f'Frecuencia Actual: {actual_counts}')


Cortes en Master_Finiciomora: [-1.001, 9.0, 16.0, 18.0, 19.0, 20.0, 22.0, 25.0, 48.0, 52.0, 299.0]
Frecuencia Esperada: [2001 2045 1211 1821 1680 3754 2048 2057 2090 2377]
Frecuencia Actual: [ 32  41   0   0   0   0 453  48   9 240]


## Analisis drifting con modelo

In [39]:
# 3- parametros para calcular ganancia del modelo
ganancia_acierto = 273000
costo_estimulo = 7000


data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001


In [40]:
# 4 - binarización de la clase ternaria

data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

In [ ]:
# 5 Especificar mes de train y test

data = data[data['foto_mes']<=202104]
df_test = data[data['foto_mes']==202106]


In [ ]:
clase_peso = data['clase_peso']
X_train = data.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =data['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']

In [ ]:
w_train = data.loc[X_train.index, 'clase_peso']


In [44]:
best_params = {'num_leaves': 40,
 'learning_rate': 0.022596031021514126,
 'min_data_in_leaf': 1692,
 'feature_fraction': 0.8060065372089812,
 'bagging_fraction': 0.8663914349386255}

In [45]:
train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)


params = {
'objective': 'binary',
'boosting_type': 'gbdt',
'first_metric_only': True,
'boost_from_average': True,
'feature_pre_filter': False,
'max_bin': 31,
'num_leaves': 0,
'learning_rate': 0,
'min_data_in_leaf': 0,
'feature_fraction': 0,
'bagging_fraction': 0,
'seed': semillas[0],
'verbose': 0
}
    #actualización con valores optuna
params.update(best_params)

if params['num_leaves'] == 0:
    print ('Error falta actualización de parámetros')


model = lgb.train(params,
                train_data,
                num_boost_round=299)



In [46]:
model_df = model.trees_to_dataframe()
model_dfz

In [47]:
model_df[model_df['split_feature'] == 'cpayroll_trx'].sort_values('threshold')

,tree_index,node_depth,node_index,left_child,right_child,parent_index,split_feature,split_gain,threshold,decision_type,missing_direction,missing_type,value,weight,count
48,0,4,0-S13,0-S20,0-L14,0-S12,cpayroll_trx,7941.270020,1.000000e-35,<=,left,None,-4.728160,6119.61000,709642
8106,102,2,102-S1,102-S4,102-S3,102-S0,cpayroll_trx,211.886993,1.000000e-35,<=,left,None,-0.007566,6325.54000,2466401
8998,113,4,113-S16,113-S31,113-L17,113-S4,cpayroll_trx,59.258999,1.000000e-35,<=,left,None,-0.011716,2649.14000,1508170
9235,116,4,116-S12,116-S26,116-L13,116-S5,cpayroll_trx,56.648602,1.000000e-35,<=,left,None,-0.011408,2576.91000,1508170
9638,122,1,122-S0,122-S1,122-S2,None,cpayroll_trx,442.329010,1.000000e-35,<=,left,None,0.000000,0.00000,4075500
9979,126,3,126-S2,126-S5,126-S4,126-S1,cpayroll_trx,156.582001,1.000000e-35,<=,left,None,-0.000830,12894.00000,1798244
10026,126,3,126-S11,126-S24,126-L12,126-S3,cpayroll_trx,70.646301,1.000000e-35,<=,left,None,-0.009593,2659.97000,1476722
10034,127,2,127-S1,127-S4,127-S2,127-S0,cpayroll_trx,138.108002,1.000000e-35,<=,left,None,0.000859,26689.50000,2570004
10103,127,4,127-S29,127-L1,127-L30,127-S10,cpayroll_trx,43.661999,1.000000e-35,<=,left,None,-0.005133,1926.04000,633010
10108,127,4,127-S33,127-L4,127-L34,127-S30,cpayroll_trx,36.843899,1.000000e-35,<=,left,None,-0.012020,1421.85000,789724


## Analisis SHAPs

Calculamos la importancia de las variables usando SHAP


In [48]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)
shap_df = pd.DataFrame(shap_values, columns = X_train.columns)

shap_importancia = pd.DataFrame(np.abs(shap_values).mean(0), columns=['SHAP Importance'])
shap_importancia['Feature'] = X_train.columns
shap_importancia.sort_values('SHAP Importance', ascending=False, inplace=True)
shap_importancia



c:\Users\Admin\.conda\envs\datascience\Lib\site-packages\shap\explainers\_tree.py:448: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn('LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray')


,SHAP Importance,Feature
46,0.426038,mpayroll
45,0.312731,cpayroll_trx
98,0.285626,ctrx_quarter
1,0.160316,foto_mes
15,0.159814,mcaja_ahorro
19,0.157954,mcuentas_saldo
9,0.132943,cproductos
24,0.126529,ctarjeta_visa_transacciones
7,0.105147,mactivos_margen
128,0.104284,Visa_msaldopesos


In [49]:
shap_importancia.columns

Index(['SHAP Importance', 'Feature'], dtype='object')

In [50]:
lista_shap_unimportant = shap_importancia.loc[shap_importancia['SHAP Importance']==0,'Feature'].to_list()

In [51]:
lista_para_sacar = []
lista_para_transformar = []

for x in lista_variables_con_drift:
    if x in lista_shap_unimportant:
        lista_para_sacar.append(x)
    if x not in lista_shap_unimportant:
        lista_para_transformar.append(x)